<a href="https://colab.research.google.com/github/Duppal147/Hackathon2025/blob/main/hackathon_pre_game_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LINK: https://www.kaggle.com/datasets/nathanlauga/nba-games

To build a machine learning model for predicting NBA game outcomes using historical Kaggle data, follow these steps:

Data Collection and Preparation
Obtain the NBA dataset from Kaggle, which includes comprehensive information on teams, players, and games28.

Clean and preprocess the data:

Remove any irrelevant or redundant features

Handle missing values

Encode categorical variables

Normalize numerical features

Create relevant features:

Calculate team performance metrics (e.g., average points scored, rebounds, assists)

Compute player statistics

Generate features based on recent performance (e.g., last 10 games)3

Feature Selection
Identify key performance indicators that influence game outcomes:

Field goal percentage

Defensive rebounds

Turnovers

Assists

Three-point shooting percentage4

Use feature importance techniques like correlation analysis or SHAP (SHapley Additive exPlanations) to select the most relevant features4.

Model Selection and Training
Split the data into training and testing sets (e.g., 80:20 ratio)3.

Choose and implement machine learning algorithms:

Logistic Regression

Random Forest Classifier

XGBoost Classifier

Support Vector Classifier

Gaussian Naïve Bayes14

Train the models using the training data.

Perform hyperparameter tuning using techniques like grid search or Bayesian optimization4.

Model Evaluation
Evaluate model performance using metrics such as:

Accuracy

Precision

Recall

F1 Score

AUC (Area Under the Curve)4

Use cross-validation techniques (e.g., 10-fold cross-validation) to ensure robust performance assessment4.

Model Refinement
Analyze feature importance to understand which factors contribute most to the predictions4.

Consider ensemble methods or stacking to combine multiple models for improved performance.

Implement techniques like rolling averages or time-based features to capture recent team performance3.

Deployment and Prediction
Select the best-performing model based on evaluation metrics.

Implement the model in a production environment.

Use the model to predict outcomes of upcoming NBA games by inputting the latest team and player statistics.

Continuously monitor and update the model with new data to maintain its accuracy over time.

By following these steps, you can create a machine learning model to predict NBA game outcomes using historical Kaggle data. Remember that the accuracy of such models typically ranges from 65% to 70%14, so while they can provide valuable insights, they are not perfect predictors of game results.




In [10]:
# Importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

In [12]:
# Function to safely load CSV files
def load_csv(filename):
    if os.path.exists(filename):
        return pd.read_csv(filename)
    else:
        print(f"Warning: {filename} not found.")
        return None

# List of expected files
file_list = ['games.csv', 'ranking.csv', 'teams.csv']

# Dictionary to store our dataframes
dataframes = {}

# Load each file
for file in file_list:
    dataframes[file.split('.')[0]] = load_csv(file)

# Now you can access each dataframe like this:
# games = dataframes['games']
# teams = dataframes['teams']
# ranking = dataframes['ranking']

# Print out which files were successfully loaded
print("Successfully loaded files:")
for key, df in dataframes.items():
    if df is not None:
        print(f"- {key}.csv")

# Print out which files were not found
print("\nFiles not found:")
for key, df in dataframes.items():
    if df is None:
        print(f"- {key}.csv")


Successfully loaded files:

Files not found:
- games.csv
- ranking.csv
- teams.csv
